# Report Concept Draft

In [ ]:
try:
    import evidently
except:
    !npm install -g yarn
    !pip install git+https://github.com/evidentlyai/evidently.git

In [ ]:
import pandas as pd
import numpy as np

from sklearn import datasets, ensemble, model_selection

from evidently import ColumnMapping
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset
from evidently.metric_preset import DataQuality
from evidently.metric_preset import CatTargetDrift
from evidently.metric_preset import NumTargetDrift
from evidently.metric_preset import RegressionPerformance
from evidently.metric_preset import ClassificationPerformance

## Prepare Datasets

In [ ]:
#Dataset for Data Quality and Integrity
adult_data = datasets.fetch_openml(name='adult', version=2, as_frame='auto')
adult = adult_data.frame

adult_ref = adult[~adult.education.isin(['Some-college', 'HS-grad', 'Bachelors'])]
adult_cur = adult[adult.education.isin(['Some-college', 'HS-grad', 'Bachelors'])]

adult_cur.iloc[:2000, 3:5] = np.nan

In [ ]:
#Dataset for regression
housing_data = datasets.fetch_california_housing(as_frame='auto')
housing = housing_data.frame

housing.rename(columns={'MedHouseVal': 'target'}, inplace=True)
housing['prediction'] = housing_data['target'].values + np.random.normal(0, 3, housing.shape[0])

housing_ref = housing.sample(n=5000, replace=False)
housing_cur = housing.sample(n=5000, replace=False)

In [ ]:
#Dataset for binary probabilistic classifcation
bcancer_data = datasets.load_breast_cancer(as_frame='auto')
bcancer = bcancer_data.frame

bcancer_ref = bcancer.sample(n=300, replace=False)
bcancer_cur = bcancer.sample(n=200, replace=False)

model = ensemble.RandomForestClassifier(random_state=1, n_estimators=10)
model.fit(bcancer_ref[bcancer_data.feature_names.tolist()], bcancer_ref.target)

bcancer_ref['pos'] = model.predict_proba(bcancer_ref[bcancer_data.feature_names.tolist()])[:, 1]
bcancer_ref['neg'] = model.predict_proba(bcancer_ref[bcancer_data.feature_names.tolist()])[:, 0]

bcancer_cur['pos'] = model.predict_proba(bcancer_cur[bcancer_data.feature_names.tolist()])[:, 1]
bcancer_cur['neg'] = model.predict_proba(bcancer_cur[bcancer_data.feature_names.tolist()])[:, 0]

In [ ]:
#Dataset for multiclass classifcation
iris_data = datasets.load_iris(as_frame='auto')
iris = iris_data.frame

iris_ref = iris.sample(n=75, replace=False)
iris_cur = iris.sample(n=75, replace=False)

model = ensemble.RandomForestClassifier(random_state=1, n_estimators=3)
model.fit(iris_ref[iris_data.feature_names], iris_ref.target)

iris_ref['prediction'] = model.predict(iris_ref[iris_data.feature_names])
iris_cur['prediction'] = model.predict(iris_cur[iris_data.feature_names])

## How to create a visual Report (ex. Dashboard)?

In [ ]:
data_drift_report = Report(metrics=[
    DataDriftPreset(),
])

data_drift_report.run(reference_data=adult_ref, current_data=adult_cur)
data_drift_report

In [ ]:
#data_drift_report.save_html('data_drift_report.html')

## How to create a json Report (ex. Profile)?

In [ ]:
data_drift_report.json()

In [ ]:
#data_drift_report.save_json('data_drift_report.json')

## How to get a python object with Report's main data?

In [ ]:
data_drift_report.as_dict()

## What Reports are avaliable?

In [ ]:
data_quality_report = Report(metrics=[
    DataQuality(),
])

data_quality_report.run(reference_data=adult_ref, current_data=adult_cur)
data_quality_report

In [ ]:
regression_performance_report = Report(metrics=[
    RegressionPerformance(),
])

regression_performance_report.run(reference_data=housing_ref.sort_index(), current_data=housing_cur.sort_index())
regression_performance_report

In [ ]:
num_target_drift_report = Report(metrics=[
    NumTargetDrift(),
])

num_target_drift_report.run(reference_data=housing_ref, current_data=housing_cur)
num_target_drift_report

In [ ]:
classification_performance_report = Report(metrics=[
    ClassificationPerformance(),
])

classification_performance_report.run(reference_data=bcancer_ref, current_data=bcancer_cur,
                                     column_mapping=ColumnMapping(prediction='pos'))
classification_performance_report

In [ ]:
cat_target_drift_report = Report(metrics=[
    CatTargetDrift(),
])

cat_target_drift_report.run(reference_data=iris_ref, current_data=iris_cur)
cat_target_drift_report